Importing libraries

The application is build in Python 3.8.6

In [2]:
import argparse
import csv
import random
from math import radians, cos, sin, sqrt, atan2

The function below parses the arguments passed to the program. It takes one optional argument, n, which is an integer. The default value of n is None and the argument parser object has an add_argument method which takes in a string and other optional parameters.

In [3]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("n", type=int, nargs='?', default=None)
    return parser.parse_args()


This function generates a list of random places. It takes in an integer n and returns a list of tuples containing the name, latitude, and longitude of each place. The number of places returned is equal to n.

* for loop will be executed 'n' times
* place_names is a list of names of cities and random.shuffle will shuffle those names
* random_places is used to append/save to list name,latitude and longitude

In [4]:
def generate_random_places(n):
    random_places = []
    place_names = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose", "Austin", "Jacksonville", "Fort Worth", "Columbus", "San Francisco", "Charlotte", "Indianapolis", "Seattle", "Denver", "Washington DC"]
    random.shuffle(place_names)
    for name in place_names:
        lat = random.randint(-90, 90)
        lon = random.randint(-180, 180)
        random_places.append((name, lat, lon))
    return random_places[:n]
        

This is a function that reads places from a csv file. It takes in an argument n which is the number of places to read. If n is not specified, it will read all the places in the csv file.

In [5]:
def read_places(n=None):
    if n:
        # Generate n random places
        random_place = generate_random_places(n)
        return random_place
    else:
        # Read from places.csv
        with open("place.csv") as f:
            reader = csv.reader(f,delimiter=';')
            next(reader)
            return [(row[0],float(row[1]), float(row[2])) for row in reader]

This function calculates the distance between two points on Earth. It takes in two tuples, each containing a name, latitude and longitude of a place (p1,p2). The function then converts the latitudes and longitudes to radians using the radians function from the math module. Then it uses the Haversine formula to calculate the distance between these two places.which is an equation important in navigation, giving great-circle distances between two points on a sphere from their longitudes and latitudes. Finally, it returns this value multiplied by 6371 km (the radius of Earth).

In [6]:
def air_distance(p1, p2):
    # Convert to radians
    lat1, lon1 = radians(p1[1]), radians(p1[2])
    lat2, lon2 = radians(p2[1]), radians(p2[2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Earth radius = 6371 km
    return 6371 * c

This is the main function of the program. It takes in an argument from the command line, which is a number that represents how many random places to generate. The function then calls read_places() and passes in n as an argument. This returns a list of tuples containing place names, latitude and longitude coordinates.

The main function then creates two empty lists: distances and seen. The for loop iterates through each tuple in places, assigning it to p1 and p2 respectively. If the tuple (p1, p2) or (p2, p1) has already been seen before, we skip it by using continue. Otherwise we add it to our set called seen so that we don't repeat ourselves later on when we encounter this pair again but with different values assigned to p1 and p2. We then call air_distance() passing in both tuples as arguments which returns the distance between them in kilometers as a float value. We append this distance along with both tuples into distances list as another tuple so that we can keep track of which pairs are closest together later on when sorting our list by distance values using key=lambda x: x[2].



In [14]:
def main():
    args = parse_args()
    n = args.n
    places = read_places(n)
    distances = []
    seen = set()
    
    for i in range(len(places)):
        for j in range(i+1, len(places)):
            p1 = places[i]
            p2 = places[j]
            if (p1, p2) in seen or (p2, p1) in seen:
                continue
            seen.add((p1, p2))
            distance = air_distance(p1, p2)
            distances.append((p1, p2, distance))
            distances.sort(key=lambda x: x[2])
            total_distance = 0
            for (p1, p2, distance) in distances:
                total_distance += distance
                print(f"{p1[0] : <20}    {p2[0]: ^15}     {distance:.2f}km.")
                avg_distance = total_distance / len(distances)
                closest_pair = min(distances, key=lambda x: abs(x[2] - avg_distance))
   
                print(f"Average distance: {avg_distance:.2f}km. Closest pair: {closest_pair[0][0]} - {closest_pair[1][0]} {closest_pair[2]:.2f} km.")
